In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import os
import torch
import torchvision
import torchaudio
from transformers import CLIPTokenizerFast, CLIPTokenizer, T5TokenizerFast
from huggingface_hub import login
from diffusers import FluxPipeline
from diffusers import StableDiffusionPipeline  # Corrected import
import ipywidgets as widgets
from IPython.display import display
from huggingface_hub import snapshot_download
from PIL import Image
import datetime
import re
int_slider = widgets.IntSlider()
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("Torchaudio Version:", torchaudio.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("MPS Available:", torch.backends.mps.is_available())  # For Apple Silicon GPU support
x = torch.rand(3, 3)
print("Tensor:", x)

huggingface_token = os.getenv("HUGGINGFACE_HUB_TOKEN")

if huggingface_token is None:
    raise ValueError("HUGGINGFACE_HUB_TOKEN environment variable is not set.")

# Authenticate with Hugging Face Hub
login(token=huggingface_token)

# ============================
# Determine the Device and Set torch_dtype Accordingly
# ============================
if torch.backends.mps.is_available():
    device = "mps"
    torch_dtype = torch.float16  # Optimal for MPS
    print("Using MPS for acceleration.")
else:
    device = "cpu"
    torch_dtype = torch.float32
    print("Using CPU for computation.")

images_dir = os.path.join(os.getcwd(), "../images")
# Create the images directory if it doesn't exist
os.makedirs(images_dir, exist_ok=True)
print(images_dir)

In [ ]:
# ============================
# Define Model Repository and Local Cache Directory
# ============================
models_dir = os.path.join(os.getcwd(), "../models")
print(models_dir)  # Should output: /Users/username/Desktop/AI-projects/kronako-text2img/notebooks/../images

In [ ]:
model_repo = "runwayml/stable-diffusion-v1-5"  
model_identifier_file = os.path.join(models_dir, "stable-diffusion-v1-5.ckpt") 

# ============================
# Check if the Model is Already Downloaded
# ============================
if not os.path.exists(model_identifier_file):
    model_local_path = snapshot_download(
        repo_id=model_repo,
        cache_dir=models_dir,
        local_dir=models_dir,
        local_dir_use_symlinks=False,  # Avoid using symlinks for better compatibility
    )
    print(f"Model downloaded to: {model_local_path}")
else:
    print(f"Model already exists. Skipping download.")


In [ ]:
# ============================
# Load the Stable Diffusion Model
# ============================
try:
    pipe = StableDiffusionPipeline.from_pretrained(
        model_repo,  # Use the model repository directly
        torch_dtype=torch_dtype,
        use_auth_token=huggingface_token  # Ensure authentication
    )
    
    # Move the pipeline to the CPU
    pipe = pipe.to(device)
    
    print("Stable Diffusion v1-5 pipeline loaded successfully.")
except Exception as e:
    print(f"Error loading the model: {e}")
    raise e

In [ ]:
def simplify_prompt(prompt):
    """
    Simplifies the prompt string to create a safe filename.
    
    Args:
        prompt (str): The original text prompt.
    
    Returns:
        str: A simplified version of the prompt suitable for filenames.
    """
    # Convert to lowercase
    simplified = prompt.lower()
    
    # Remove non-alphanumeric characters (except spaces)
    simplified = re.sub(r'[^a-z0-9\s]', '', simplified)
    
    # Replace multiple spaces with a single space
    simplified = re.sub(r'\s+', ' ', simplified)
    
    # Replace spaces with underscores
    simplified = simplified.replace(' ', '_')
    
    return simplified


In [ ]:
def get_timestamp():
    """
    Generates a current timestamp.
    
    Returns:
        str: Timestamp in 'YYYYMMDD_HHMMSS' format.
    """
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
def create_filename(prompt, extension=".png", max_length=50, directory=images_dir):
    """
    Creates a dynamic filename based on the prompt and current timestamp.
    Ensures the filename is unique by checking existing files.
    
    Args:
        prompt (str): The original text prompt.
        extension (str): File extension (default is '.png').
        max_length (int): Maximum length for the simplified prompt part of the filename.
        directory (str): Directory where the image will be saved.
    
    Returns:
        str: A dynamic and unique filename with directory path.
    """
    simplified_prompt = simplify_prompt(prompt)
    
    # Truncate if necessary to prevent overly long filenames
    if len(simplified_prompt) > max_length:
        simplified_prompt = simplified_prompt[:max_length]
    
    timestamp = get_timestamp()
    filename = f"{simplified_prompt}_{timestamp}{extension}"
    
    # Combine directory and filename
    filepath = os.path.join(directory, filename)
    
    # Ensure filename is unique
    while os.path.exists(filepath):
        timestamp = get_timestamp()
        filename = f"{simplified_prompt}_{timestamp}{extension}"
        filepath = os.path.join(directory, filename)
    
    return filepath


In [ ]:
def generate_sketch(prompt, num_inference_steps=15, guidance_scale=7.5, height=512, width=512):
    """
    Generates a sketch based on the provided text prompt.
    
    Args:
        prompt (str): The text prompt describing the desired sketch.
        num_inference_steps (int): Number of inference steps (trade-off between speed and quality).
        guidance_scale (float): Controls how much the model follows the prompt (higher = more adherence).
        height (int): Image height in pixels.
        width (int): Image width in pixels.
    
    Returns:
        PIL.Image: Generated sketch image.
    """
    try:
        # Generate the image
        result = pipe(
            prompt=prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            height=height,
            width=width,
        )
        
        # Retrieve the image
        image = result.images[0]
        return image
    except Exception as e:
        print(f"Error during image generation: {e}")
        return None


In [ ]:
# ============================
# Define a Function to Generate and Save Sketches
# ============================

def generate_and_save_sketch(prompt, num_inference_steps=15, guidance_scale=7.5, height=512, width=512):
    """
    Generates a sketch based on the provided text prompt and saves it with a dynamic filename.
    
    Args:
        prompt (str): The text prompt describing the desired sketch.
        num_inference_steps (int): Number of inference steps.
        guidance_scale (float): Controls how much the model follows the prompt.
        height (int): Image height in pixels.
        width (int): Image width in pixels.
    
    Returns:
        None
    """
    # Generate the sketch
    print("Generating sketch...")
    sketch_image = generate_sketch(prompt, num_inference_steps=num_inference_steps, height=height, width=width)
    
    if sketch_image:
        # Display the image
        sketch_image.show()
        
        # Create a dynamic filename
        output_path = create_filename(prompt)
        
        # Save the image locally
        sketch_image.save(output_path)
        print(f"Sketch generated and saved successfully at {output_path}.")
    else:
        print("Failed to generate the sketch.")



In [ ]:
prompt = "Bridging Shadows A bridge spanning a river, with shadowy figures on one side and brightly lit figures on the other. The water below reflects both sides, blending them into one indistinguishable image. Themes: Race and identity, unity and division, self-reflection. Why Lowell?: The city’s many bridges symbolize connection yet often divide communities by race and class."


In [ ]:
generate_and_save_sketch(prompt, num_inference_steps=15, height=512, width=512)